In [43]:
import pickle
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
#from sklearn import datasets
from sklearn.datasets import load_iris
from moviepy.editor import *

success = True
x_axis, y_axis = 0, 0
count = 0
# mode: 0 or 1, for different colors
color_mode = 0
img_list = []

cap = cv2.VideoCapture("../DemoVideo/test.mp4")
total_frame = cap.get(7)
speed = cap.get(5)
print(total_frame, speed)

while success and count < total_frame:
    success, frame = cap.read()
    pickle_path = "../out/res/" + str(count) + ".pkl"
    if os.path.exists(pickle_path) == False:
        success = False
    outeample = pickle.load(open(pickle_path, "rb"))
    count = count + 1
    # GaussianBlur to Blur the image
    frame = cv2.GaussianBlur(frame, (5, 5), 1)
    outeample_numpy = np.array(outeample['human_bboxes'])
    center_slide = []
    for per_human_box in outeample_numpy:
        x_min, y_min, x_max, y_max = per_human_box
        center_x = (int)(x_min + x_max) // 2
        center_y = (int)(y_min + y_max) // 2
        mid_point = [
            [center_y - 1, center_x - 1], [center_y - 1,
                                           center_x], [center_y - 1, center_x + 1],
            [center_y, center_x - 1], [center_y,
                                       center_x], [center_y, center_x + 1],
            [center_y + 1, center_x - 1], [center_y + 1,
                                           center_x], [center_y + 1, center_x + 1]
        ]
        center_slide.append(mid_point)
    color_pair = []
    for mid_point in center_slide:
        color_list = []
        for coordinate in mid_point:
            color = frame[coordinate[0], coordinate[1]]
            color_list.append(color)
        color_list = np.average(color_list, axis=0)
        color_pair.append(color_list)
    for idx in range(len(color_pair)):
        color_pair[idx] = color_pair[idx].tolist()
    color_pair =  np.array(color_pair)
    estimator = KMeans(n_clusters=2)
    estimator.fit(color_pair)
    label_pred = estimator.labels_
    x0 = color_pair[label_pred == 0]
    x1 = color_pair[label_pred == 1]
    x0_avg = np.average(x0)
    x1_avg = np.average(x1)
    if x0_avg < x1_avg:
        result = [x0, x1]
    else:
        result = [x1, x0]
    cluster = []
    for i in range(len(result)):
        cluster_per = []
        for result_per in result[i]:
            for idx in range(len(color_pair)):
                if np.sum(np.abs(result_per - color_pair[idx])) < 0.5:
                    cluster_per.append(idx)
        cluster.append(cluster_per)
    img = np.zeros((len(frame), len(frame[0])))
    for idx in cluster[color_mode]:
        fontsize = 2
        x_space = center_slide[idx][4][1]-19
        y_space = center_slide[idx][4][0]+19
        if idx >= 10:
            fontsize = 1
            y_space = y_space - 3
        cv2.circle(
            img, (center_slide[idx][4][1], center_slide[idx][4][0]), 30, (255, 255, 255), 2)
        cv2.putText(img, str(idx), (x_space, y_space),
                    cv2.FONT_HERSHEY_SIMPLEX, fontsize, (255, 255, 255), 2)
    img = np.dstack(3 * [img]).astype('uint8')
    img_list.append(img)

new_clip = ImageSequenceClip(img_list, fps=(int)(speed))
vis_path = "test_out.mp4"
new_clip.write_videofile(vis_path)


146.0 25.0
Moviepy - Building video test_out.mp4.
Moviepy - Writing video test_out.mp4



Moviepy - Done !
Moviepy - video ready test_out.mp4


In [14]:
import numpy as np
set_data = np.ones(100)
for i in range(100):
    set_data[i] = i
set_data = np.dstack(3 * [set_data]).astype('uint8').reshape(10, -1, 3)
print(set_data.shape)

(10, 10, 3)


In [65]:
# print(set_data[1:5, 3:6]) # [a, b)
clip = set_data[1:5, 3:6] # [a, b)
clip = clip.reshape(1, -1, 3)[0]
print(clip)
seq = clip[:,1].argsort() #按照第3列对行排序
print(seq)

[[13 13 13]
 [14 14 14]
 [15 15 15]
 [23 23 23]
 [24 24 24]
 [25 25 25]
 [33 33 33]
 [34 34 34]
 [35 35 35]
 [43 43 43]
 [44 44 44]
 [45 45 45]]
[ 0  1  2  3  4  5  6  7  8  9 10 11]


In [26]:
import pickle
data = pickle.load(open("/Disk4/xudong/HAKE-Action-Torch-Activity2Vec/out/res/89.pkl", "rb"))

In [27]:
print(data.keys())

dict_keys(['human_bboxes', 'part_bboxes', 'keypoints', 'human_scores', 'skeletons', 'f_pasta', 'p_pasta', 'p_verb'])


In [30]:
print(data['human_bboxes'][13])

tensor([  0.0000, 635.5928,  39.9791, 758.7194])
